In [4]:
import json
import pandas as pd
from collections import Counter
import numpy as np


pd.set_option('display.max_rows', 1000)

#file contains list of interested genes type json: this can be changed
#filename_s = 'file\extract\CPC013_HCC515_6H_BRD-K05104363_10.0.json'

filename_u = 'Up_PY010.csv'
filename_d = 'Down_PY010.csv'

#with open(filename_s, "rb") as data_in_s:
  #  data_s = json.load(data_in_s)

df_u =  pd.read_csv(filename_u);
df_d =  pd.read_csv(filename_d);
upgene= df_u['gene']
dngene= df_d['gene']

my_list_compare = ['genes']
my_list_compare_w = []
my_list_compare_u = []
type_gene = []
gene = []
i = 0


for item_s in upgene:
    my_list_compare_w.append(item_s)
    my_list_compare_u.append(item_s)

for item in dngene:
    my_list_compare_w.append(item)


gene_list = Counter(my_list_compare_w).keys()
gene_count =Counter(my_list_compare_w).values()
gene_cu =Counter(my_list_compare_u).values()


gene = list(gene_list)
genec = list(gene_count)
geneu = list(gene_cu)

gene_b = []
gene_u = []
gene_d = []
gene_a = []

for x in range(0,len(gene_count)):
    if genec[x] != 2:
        if x  <  sum(geneu): 
            type_gene.append('up gene')
            gene_u.append(gene[x])
            gene_a.append(gene[x])
        else: 
            type_gene.append('down gene')
            gene_d.append(gene[x])
            gene_a.append(gene[x])
    else:    
        type_gene.append('both')
        gene_b.append(gene[x])
        gene_a.append(gene[x])
#for item in dngene:
    #my_set = set(item)
    #my_new_list_dn = list(my_set)
    #for i in my_new_list_dn:
  #  my_list_compare.append(item)
    #my_list_compare_w.append(item)
    
df_pd = pd.DataFrame(data = type_gene, columns = ['type'])
df_pd['gene'] = gene_a
df_pd

,type,gene
0,up gene,TIMP4
1,up gene,SNORA13
2,up gene,GNG13
3,up gene,PLCE1
4,up gene,LSP1
5,up gene,TPRXL
6,up gene,LGALS4
7,up gene,KCNAB3
8,up gene,BEND6
9,up gene,OGN


In [8]:
from bsonstream import KeyValueBSONInput
import csv
import bson 
import json
from bson import ObjectId
import numpy as np
from string import punctuation

def strip_punctuation(s):
    return ''.join(c for c in s if c not in punctuation)

with open('file\cpcd-gse70138.bson', 'rb') as f:
    #name of .csv file
    with open('name_do_not_have_list_gene.csv', 'w') as outfile:
        write = csv.writer(outfile)
        #write.writerow(my_list_compare)
        stream = KeyValueBSONInput(fh=f)
        for dict_data in stream:
            del dict_data['_id']
            #name of row in .csv
            name = str(dict_data['batch'])+'_' +str(dict_data['pert_id'])+'_' +str(dict_data['pert_dose'])
            #get rid of file that does not contian a list of genes
                #get rid of file that does not contian a list of genes
           # if dict_data.get('upGenes') != None:
             #   output = np.zeros(len(gene_a), dtype=object);
                #output[0] = name
                #index = 0
                #comparing interested up regulated gene set and file from .bson
               # for i,b in zip(gene_a,type_gene):
                 #   for j in dict_data.get('upGenes'):
                   #     if i == j and (b == 'up gene' or b == 'both'):
                     #       output[index] = '1'
                    #for y in dict_data.get('dnGenes'):
                      #  if i == y and (b == 'down gene' or b == 'both'):
                        #    output[index] = '-1'
                    #index = index +1
                    #index = index +1
                #write.writerow(output)
               # df_pd[name] = output
            if dict_data.get('upGenes') == None:
                write.writerow([name])

C:\Users\Arada\Miniconda\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: generator 'KeyValueBSONInput.reads' raised StopIteration


In [244]:
df_pd

,type,gene,CPC001_HA1E_24H_BRD-A19500257_10.0,CPC001_HA1E_24H_BRD-A82329351_10.0,CPC001_HA1E_24H_BRD-K02607075_10.0,CPC001_HA1E_24H_BRD-A49906757_10.0,CPC001_HA1E_24H_BRD-K85242180_10.0,CPC001_HA1E_24H_BRD-K78844995_10.0,CPC001_HA1E_24H_BRD-A45140972_10.0,CPC001_HA1E_24H_BRD-K63938928_10.0,...,CPC001_HCC515_24H_BRD-K23623876_10.0,CPC001_HCC515_24H_BRD-A68281735_10.0,CPC001_HCC515_24H_BRD-K63150726_10.0,CPC001_HCC515_24H_BRD-A70268693_10.0,CPC001_HCC515_24H_BRD-K51276371_10.0,CPC001_HCC515_24H_BRD-K64670467_10.0,CPC001_HCC515_24H_BRD-K84595254_10.0,CPC001_HCC515_24H_BRD-A22707317_10.0,CPC001_HCC515_24H_BRD-A82329351_10.0,CPC001_HCC515_24H_BRD-K71860425_10.0
0,up gene,TIMP4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,up gene,SNORA13,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,up gene,GNG13,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,up gene,PLCE1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,up gene,LSP1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,up gene,TPRXL,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,up gene,LGALS4,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,1
7,up gene,KCNAB3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,up gene,BEND6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,up gene,OGN,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [271]:
df_pd
df_pd.to_csv('PY010_table_with_type2.csv',  encoding='utf-8')

In [ ]:
from bsonstream import KeyValueBSONInput
import csv
import bson 
import json
from bson import ObjectId
import numpy as np
from string import punctuation

def strip_punctuation(s):
    return ''.join(c for c in s if c not in punctuation)

with open('file\cpcd-gse70138.bson', 'rb') as f:
    #name of .csv file
    with open('name2.csv', 'w') as outfile:
        write = csv.writer(outfile)
        #write.writerow(my_list_compare)
        stream = KeyValueBSONInput(fh=f)
        for dict_data in stream:
            del dict_data['_id']
            #name of row in .csv
            name = str(dict_data['batch'])+'_' +str(dict_data['pert_id'])+'_' +str(dict_data['pert_dose'])
            #get rid of file that does not contian a list of genes
            if dict_data.get('upGenes') != None:
                output = np.zeros(len(my_list_compare_w), dtype=object);
                #output[0] = name
                index = 0
                #comparing interested up regulated gene set and file from .bson
                for i in upgene:
                    for j in dict_data.get('upGenes'):
                        if i == j:
                            output[index] = '1'
                    index = index +1
                #comparing interested up regulated gene set and file from .bson
                for x in dngene:
                    for y in dict_data.get('dnGenes'):
                        if x == y:
                            output[index] = '-1'
                    index = index +1
                #write.writerow(output)
                df_pd[name] = output
